## Version: With Opt R Regression , Feature selection and OOF OPT R

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin
from numba import jit

import joblib
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import json
pd.set_option('display.max_columns', 1000)

/kaggle/input/data-science-bowl-model/model_1.pkl
/kaggle/input/data-science-bowl-model/fold_feats.pkl
/kaggle/input/data-science-bowl-model/__results__.html
/kaggle/input/data-science-bowl-model/model_3.pkl
/kaggle/input/data-science-bowl-model/model_2.pkl
/kaggle/input/data-science-bowl-model/custom.css
/kaggle/input/data-science-bowl-model/__notebook__.ipynb
/kaggle/input/data-science-bowl-model/features.npy
/kaggle/input/data-science-bowl-model/model_0.pkl
/kaggle/input/data-science-bowl-model/model_4.pkl
/kaggle/input/data-science-bowl-model/top_features.npy
/kaggle/input/data-science-bowl-model/__output__.json
/kaggle/input/data-science-bowl-model/optR_fulltrain.pkl
/kaggle/input/data-science-bowl-model/__results___files/__results___69_0.png
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/sample_submission.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-201

In [2]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=2000


In [3]:
from collections import Counter
from sklearn import linear_model
from functools import partial
from sklearn import metrics
import json
import scipy as sp

# Objective

* In the last notebook we create our baseline model including a feature selection part. 
* Cohen cappa score of 0.456 (lb) with a local cv score of 0.529
* In this notebook we are going to add more features and remove others that i think they overfitt the train set and then check if our local cv score improve.
* Next, we will check if this improvement aligns with the lb.

# Notes
* Check the distribution of the target variable of the out of folds score and the prediction distribution. A good model should more or less have the same distribution.

In [4]:
n_splits=5

In [5]:
import os
model_path ='../input/data-science-bowl-model/'
# model_path ='../input/data-science-bowl-world-models/'
for i in range(n_splits):
    filename= 'model_'+str(i)+'.pkl'
    stat= os.stat(model_path+filename)
    size= stat.st_size
    mod_time= stat.st_mtime
    print('{0} size: {1} modified time: {2}'.format(filename,size,mod_time))

model_0.pkl size: 795095 modified time: 1576433119.5238187
model_1.pkl size: 925485 modified time: 1576433119.5428205
model_2.pkl size: 957612 modified time: 1576433119.6118262
model_3.pkl size: 930908 modified time: 1576433119.511818
model_4.pkl size: 946031 modified time: 1576433119.5598218


In [6]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

    print('Reading train_labels.csv file....')
    train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
    print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

    print('Reading specs.csv file....')
    specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, train_labels, specs, sample_submission

In [7]:
def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = sorted(list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique())))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = sorted(list(set(train['title'].unique()).union(set(test['title'].unique()))))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = sorted(list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_event_id = sorted(list(set(train['event_id'].unique()).union(set(test['event_id'].unique()))))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = sorted(list(set(train[train['type'] == 'Assessment']['title'].value_counts().index)\
                         .union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code

In [8]:
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_text = 'type_'
    user_activities_count = {user_activities_text + value: 0 for value in ['Clip','Activity','Assessment','Game']}
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups_text = 'accuracy_group_'
    accuracy_groups = {accuracy_groups_text + value: 0 for value in ['0','1','2','3']}

    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    
    event_code_text = 'ev_code_'
    event_code_count: Dict[str, int] = {event_code_text + str(ev): 0 for ev in list_of_event_code}
    event_id_text = 'ev_id_'
    event_id_count: Dict[str, int] = {event_id_text + str(eve): 0 for eve in list_of_event_id}
    title_text = 'title_'
    title_count: Dict[str, int] = {title_text + str(eve): 0 for eve in activities_labels.values()} 
    title_event_code_text = 'titleevcode_'
    title_event_code_count: Dict[str, int] = {title_event_code_text + str(t_eve): 0 for t_eve in all_title_event_code}
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            
            features['session_world'] = session['world'].iloc[0]
            
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['game_session'] = session['game_session'].iloc[0]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[accuracy_groups_text + str(features['accuracy_group'])] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
            
        def getkey(col,value):
            
            switcher = { 
                    "event_code": event_code_text, 
                    "event_id": event_id_text, 
                    "title": title_text, 
                    "title_event_code": title_event_code_text, 
                } 
            return switcher.get(col, "nothing") + str(value)
        
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    key = getkey(col,x)
                    counter[key] += num_of_session_count[k]
#                     counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[user_activities_text + str(session_type)] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        if len(all_assessments)==0:
            return None
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [9]:
def get_compiled_data(data,test_set=False,total=17000):
    compiled_data = []
#     for i, (ins_id, user_sample) in tqdm(enumerate(data.groupby('installation_id', sort = False)), total = total):
    for ins_id, user_sample in tqdm(data.groupby('installation_id', sort = False), total = total):
        
        proc_data = get_data(user_sample, test_set = test_set)
        #for test set, proc_data will be dataframe and for train, it will be list type
        if test_set:
            if proc_data is not None:
                compiled_data.append(proc_data)
        else:
            compiled_data += proc_data
        
    reduce_data = pd.DataFrame(compiled_data)
    categoricals = ['session_title']
    reduce_data.columns = reduce_data.columns.map(str)
    return reduce_data, categoricals


In [10]:
def predict(reduce_test, sample_submission, y_pred, title_suffix=''):
    sample_submission['accuracy_group'] = y_pred.argmax(axis = 1)
    sample_submission.to_csv('submission'+title_suffix+'.csv', index = False)
    print(sample_submission['accuracy_group'].value_counts(normalize = True))

In [11]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [12]:
@jit
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

In [13]:
# improved
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0
    
    def _kappa_loss(self, coef, X, y):
        preds = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, preds)
#         return -cohen_kappa_score(y, preds, weights = 'quadratic')
    
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')
    
    def predict(self, X, coef):
        preds = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return preds
    
    def coefficients(self):
        return self.coef_['x']

In [14]:
def replacezerocols_withnan(data):
    zero_cols =[]
    for col in data.columns:
        if data[col].nunique()==1:
            zero_cols += [col]
    print(len(zero_cols))
    print()
    print(zero_cols)
    
    if 'session_world' in zero_cols:
        zero_cols.remove('session_world')
    
    for col in zero_cols:
        data[col]=np.nan
    return data

In [15]:
%%time
# function that creates more features
def preprocess(reduce_data,test_set=False):
    for df in [reduce_data]:
        df['installation_session_count'] = df.groupby(['installation_id'])['type_Clip'].transform('count')
        df['installation_duration_mean'] = df.groupby(['installation_id'])['duration_mean'].transform('mean')
        #df['installation_duration_std'] = df.groupby(['installation_id'])['duration_mean'].transform('std')
        df['installation_title_nunique'] = df.groupby(['installation_id'])['session_title'].transform('nunique')
        
        event_code_cols = [col for col in df.columns if 'ev_code_' in col]
        df['sum_event_code_count'] = df[event_code_cols].sum(axis = 1)
        df['installation_event_code_count_mean'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('mean')
#         df['installation_event_code_count_std'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('std')
        
        df['sum_session'] = df['ev_code_2000']
        #perform data to percent conversion
        df= preproc_convert_to_percent(df)
        #after percent conversion

        event_id_cols = [col for col in df.columns if 'ev_id_' in col]
        df['std_event_code_count'] = df[event_code_cols].std(axis = 1)
        df['std_event_id_count'] = df[event_id_cols].std(axis = 1)
        
     
    features = None
    if not test_set:
#         features = reduce_data.columns
        features = reduce_data.loc[(reduce_data.sum(axis=1) != 0), (reduce_data.sum(axis=0) != 0)].columns # delete useless columns
        features = [x for x in features if x not in ['accuracy_group','game_session' ,\
                                                     'installation_id']] + ['acc_' + title for title in assess_titles]
   
    return reduce_data, features

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.6 µs


In [16]:
def convert_to_percent_bycol(groupcols,data):
    col_sum = data[groupcols].sum(axis=1)
    for col in tqdm(groupcols):
        data[col] = data[col] / col_sum
        
    return data

In [17]:
def preproc_convert_to_percent(data):
    ev_code_cols = [col for col in data.columns if 'ev_code_' in col]
    print(len(ev_code_cols))
    ev_id_cols = [col for col in data.columns if 'ev_id_' in col]
    print(len(ev_id_cols))
    title_cols = [col for col in data.columns if col.startswith('title_')]
    print(len(title_cols))
    title_event_cols = [col for col in data.columns if 'titleevcode_' in col]
    print(len(title_event_cols))
    type_cols = [col for col in data.columns if 'type_' in col]
    print(len(type_cols))
    acc_gp_cols = [col for col in data.columns if 'accuracy_group_' in col]
    print(len(acc_gp_cols))
    
    data = convert_to_percent_bycol(ev_code_cols,data)
    data = convert_to_percent_bycol(ev_id_cols,data)
    data = convert_to_percent_bycol(title_cols,data)
    data = convert_to_percent_bycol(title_event_cols,data)
    data = convert_to_percent_bycol(type_cols,data)
    data = convert_to_percent_bycol(acc_gp_cols,data)  
    
    return data

In [18]:
%%time
# read data
train, test, train_labels, specs, sample_submission = read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train, test, train_labels)

Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns
CPU times: user 2min 6s, sys: 18.2 s, total: 2min 25s
Wall time: 2min 24s


In [19]:
del train;gc.collect()

477

In [20]:
# sample_ids = test['installation_id'].unique()[0:10]
# test_sample = test[test['installation_id'].isin(sample_ids)]
# print(test_sample.shape)

# sample_ids_train = train['installation_id'].unique()[0:10]
# train_sample = train[train['installation_id'].isin(sample_ids_train)]
# print(train_sample.shape)

In [21]:
%%time
# tranform function to get the train and test set
reduce_test, categoricals = get_compiled_data(test, test_set=True,total=1000)
print(reduce_test.shape)
# print(list(reduce_test.columns))  


(1000, 897)
CPU times: user 49.7 s, sys: 1.74 s, total: 51.4 s
Wall time: 49.4 s


In [22]:
del test
gc.collect()

350

In [23]:
#Factors of classes
class_factors = train_labels['accuracy_group'].value_counts(normalize=True).sort_index().values
print(class_factors)

[0.23906162 0.13629169 0.12464669 0.5       ]


In [24]:
%%time
# call feature engineering function
reduce_test, dummy_feats = preprocess(reduce_test,test_set=True)

42
384
44
403
4
4



CPU times: user 1.64 s, sys: 44 ms, total: 1.68 s
Wall time: 1.49 s


In [25]:
features = list(np.load(model_path+'features.npy'))
print(len(features))

478


In [26]:
print(len(features))

478


In [27]:
%%time
reduce_test[features].head()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 7.75 ms


,type_Clip,type_Activity,type_Assessment,type_Game,acc_Bird Measurer (Assessment),acc_Cart Balancer (Assessment),acc_Cauldron Filler (Assessment),acc_Chest Sorter (Assessment),acc_Mushroom Sorter (Assessment),ev_code_2000,ev_code_2010,ev_code_2020,ev_code_2025,ev_code_2030,ev_code_2035,ev_code_2040,ev_code_2050,ev_code_2060,ev_code_2070,ev_code_2075,ev_code_2080,ev_code_2081,ev_code_2083,ev_code_3010,ev_code_3020,ev_code_3021,ev_code_3110,ev_code_3120,ev_code_3121,ev_code_4010,ev_code_4020,ev_code_4021,ev_code_4022,ev_code_4025,ev_code_4030,ev_code_4031,ev_code_4035,ev_code_4040,ev_code_4045,ev_code_4050,ev_code_4070,ev_code_4080,ev_code_4090,ev_code_4095,ev_code_4100,ev_code_4110,ev_code_4220,ev_code_4230,ev_code_4235,ev_code_5000,ev_code_5010,ev_id_0086365d,ev_id_00c73085,ev_id_022b4259,ev_id_02a42007,ev_id_0330ab6a,ev_id_0413e89d,ev_id_04df9b66,ev_id_05ad839b,ev_id_06372577,ev_id_070a5291,ev_id_08fd73f3,ev_id_08ff79ad,ev_id_0a08139c,ev_id_0d18d96c,ev_id_0d1da71f,ev_id_0db6d71d,ev_id_1325467d,ev_id_1340b8d7,ev_id_1375ccb7,ev_id_14de4c5d,ev_id_155f62a4,ev_id_1575e76c,ev_id_15a43e5b,ev_id_15ba1109,ev_id_15eb4a7d,ev_id_15f99afc,ev_id_160654fd,ev_id_16667cc5,ev_id_16dffff1,ev_id_17113b36,ev_id_19967db1,ev_id_1996c610,ev_id_1af8be29,ev_id_1bb5fbdb,ev_id_1beb320a,ev_id_1c178d24,ev_id_1cc7cfca,ev_id_1cf54632,ev_id_1f19558b,ev_id_222660ff,ev_id_2230fab4,ev_id_250513af,ev_id_25fa8af4,ev_id_262136f4,ev_id_26a5a3dd,ev_id_26fd2d99,ev_id_27253bdc,ev_id_28520915,ev_id_28a4eb9a,ev_id_28ed704e,ev_id_28f975ea,ev_id_29bdd9ba,ev_id_29f54413,ev_id_2a444e03,ev_id_2a512369,ev_id_2b058fe3,ev_id_2b9272f4,ev_id_2c4e6db0,ev_id_2dc29e21,ev_id_2dcad279,ev_id_2fb91ec1,ev_id_30614231,ev_id_30df3273,ev_id_31973d56,ev_id_3323d7e9,ev_id_33505eae,ev_id_3393b68b,ev_id_363c86c9,ev_id_363d3849,ev_id_36fa3ebe,ev_id_37937459,ev_id_37c53127,ev_id_37db1c2f,ev_id_37ee8496,ev_id_38074c54,ev_id_392e14df,ev_id_3a4be871,ev_id_3afb49e6,ev_id_3afde5dd,ev_id_3b2048ee,ev_id_3babcb9b,ev_id_3bb91ced,ev_id_3bb91dda,ev_id_3bf1cf26,ev_id_3bfd1a65,ev_id_3ccd3f02,ev_id_3d0b9317,ev_id_3d63345e,ev_id_3d8c61b0,ev_id_3dcdda7f,ev_id_3ddc79c3,ev_id_3dfd4aa4,ev_id_3edf6747,ev_id_3ee399c3,ev_id_44cb4907,ev_id_45d01abe,ev_id_461eace6,ev_id_46b50ba8,ev_id_46cd75b4,ev_id_47026d5f,ev_id_47efca07,ev_id_47f43a44,ev_id_48349b14,ev_id_4901243f,ev_id_499edb7c,ev_id_49ed92e9,ev_id_4a09ace1,ev_id_4a4c3d21,ev_id_4b5efe37,ev_id_4bb2f698,ev_id_4c2ec19f,ev_id_4d6737eb,ev_id_4d911100,ev_id_4e5fc6f5,ev_id_4ef8cdd3,ev_id_51102b85,ev_id_51311d7a,ev_id_5154fc30,ev_id_5290eab1,ev_id_532a2afb,ev_id_5348fd84,ev_id_53c6e11a,ev_id_55115cbd,ev_id_562cec5f,ev_id_565a3990,ev_id_56817e2b,ev_id_56bcd38d,ev_id_56cd3b43,ev_id_5859dfb6,ev_id_587b5989,ev_id_58a0de5c,ev_id_598f4598,ev_id_5a848010,ev_id_5b49460a,ev_id_5be391b5,ev_id_5c2f29ca,ev_id_5c3d2b2f,ev_id_5d042115,ev_id_5de79a6a,ev_id_5e109ec3,ev_id_5e3ea25a,ev_id_5e812b27,ev_id_5f0eb72c,ev_id_5f5b2617,ev_id_6043a2b4,ev_id_6077cc36,ev_id_6088b756,ev_id_63f13dd7,ev_id_65a38bf7,ev_id_65abac75,ev_id_67439901,ev_id_67aa2ada,ev_id_69fdac0a,ev_id_6aeafed4,ev_id_6bf9e3e1,ev_id_6c517a88,ev_id_6c930e6e,ev_id_6cf7d25c,ev_id_6d90d394,ev_id_6f445b57,ev_id_6f4adc4b,ev_id_6f4bd64e,ev_id_6f8106d9,ev_id_7040c096,ev_id_709b1251,ev_id_71e712d8,ev_id_71fe8f75,ev_id_731c0cbe,ev_id_736f9581,ev_id_7372e1a5,ev_id_73757a5e,ev_id_7423acbc,ev_id_74e5f8a7,ev_id_7525289a,ev_id_756e5507,ev_id_763fc34e,ev_id_76babcde,ev_id_77261ab5,ev_id_77c76bc5,ev_id_77ead60d,ev_id_792530f8,ev_id_795e4a37,ev_id_7961e599,ev_id_7ab78247,ev_id_7ad3efc6,ev_id_7cf1bc53,ev_id_7d093bf9,ev_id_7d5c30a2,ev_id_7da34a02,ev_id_7dfe6d8a,ev_id_7ec0c298,ev_id_7f0836bf,ev_id_804ee27f,ev_id_828e68f9,ev_id_832735e1,ev_id_83c6c409,ev_id_84538528,ev_id_84b0e0c8,ev_id_857f21c0,ev_id_85d1b0de,ev_id_85de926c,ev_id_86ba578b,ev_id_86c924c4,ev_id_87d743c1,ev_id_884228c8,ev_id_88d4a5be,ev_id_895865f3,ev_id_89aace00,ev_id_8ac7cce4,ev_id_8af75982,ev_id_8b757ab8,ev_id_8d748b58,ev_id_8d7e386c,ev_id_8d84fa81,ev_id_8f094001,ev_id_8fee50e2,ev_id

In [28]:
# %%time


# temp_path= '../input/data-science-bowl-model/'
# ref_features = list(np.load(temp_path+'features.npy'))

# reduce_test_ref = reduce_test_all.copy()
# reduce_test_ref.columns = [col.replace('all_','') for col in reduce_test_ref]

# print(len(ref_features))
# y_pred = np.zeros((reduce_test.shape[0], 1))

# #load model check
# for i in range(n_splits):
#     model_cur = joblib.load(temp_path+'model1_'+str(i)+'.pkl')
#     y_pred += model_cur.predict(reduce_test_ref[ref_features],
#                                 num_iteration=model_cur.best_iteration).reshape(-1, 1) / n_splits

# y_pred = [item for sublist in y_pred for item in sublist]
# print(pd.Series(y_pred).describe())

# predictions = np.array(y_pred)
# predictions[predictions <= 1.12232214] = 0
# predictions[np.where(np.logical_and(predictions > 1.12232214, predictions <= 1.73925866))] = 1
# predictions[np.where(np.logical_and(predictions > 1.73925866, predictions <= 2.22506454))] = 2
# predictions[predictions > 2.22506454] = 3

# predictions= predictions.astype(int)
# predictions_ref = predictions.copy()
# pd.Series(predictions).value_counts(normalize=True)

In [29]:
%%time
# y_pred = np.zeros((len(reduce_test), 4))
y_pred = np.zeros((reduce_test.shape[0], 1))

#load model check
for i in range(n_splits):
    model_cur = joblib.load(model_path+'model_'+str(i)+'.pkl')
    y_pred += model_cur.predict(reduce_test[features],
                                num_iteration=model_cur.best_iteration).reshape(-1, 1) / n_splits

y_pred = [item for sublist in y_pred for item in sublist]
pd.Series(y_pred).describe()

CPU times: user 1.11 s, sys: 4 ms, total: 1.12 s
Wall time: 665 ms


count    1000.000000
mean        1.887520
std         0.676858
min         0.002860
25%         1.471765
50%         2.066430
75%         2.407100
max         2.853664
dtype: float64

In [30]:
# %%time
# y_pred1 = np.zeros((reduce_test.shape[0], 1))
# #load model check
# for i in range(n_splits):
#     model_cur = joblib.load(model_path+'model1_'+str(i)+'.pkl')
#     y_pred1 += model_cur.predict(reduce_test[features],
#                                 num_iteration=model_cur.best_iteration).reshape(-1, 1) / n_splits

    
# y_pred1 = np.array([item for sublist in y_pred1 for item in sublist])
# print(pd.Series(y_pred1).describe())

# y_pred2 = np.zeros((reduce_test.shape[0], 1))
# #load model check
# for i in range(n_splits):
#     model_cur = joblib.load(model_path+'model2_'+str(i)+'.pkl')
#     y_pred2 += model_cur.predict(reduce_test[fold_feats_removefeats[i]],
#                                 num_iteration=model_cur.best_iteration).reshape(-1, 1) / n_splits
    
# y_pred2 = np.array([item for sublist in y_pred2 for item in sublist])
# print(pd.Series(y_pred2).describe())

# y_pred3 = np.zeros((reduce_test.shape[0], 1))
# #load model check
# for i in range(n_splits):
#     model_cur = joblib.load(model_path+'model3_'+str(i)+'.pkl')
#     y_pred3 += model_cur.predict(reduce_test[fold_feats_addfeats[i]],
#                                 num_iteration=model_cur.best_iteration).reshape(-1, 1) / n_splits
    
# y_pred3 = np.array([item for sublist in y_pred3 for item in sublist])
# print(pd.Series(y_pred3).describe())

# w=[0.5,0.5,0]
# # w=[0.0001,1,0.001]
# # w=[1,0.1,0.0]
# total = np.array(w).sum()
# y_pred = (w[0]*y_pred1 + w[1]*y_pred2 + w[2]*y_pred3) / total 
# print(pd.Series(y_pred).describe())

In [31]:
predictions = np.array(y_pred)
predictions[predictions <= 1.12232214] = 0
predictions[np.where(np.logical_and(predictions > 1.12232214, predictions <= 1.73925866))] = 1
predictions[np.where(np.logical_and(predictions > 1.73925866, predictions <= 2.22506454))] = 2
predictions[predictions > 2.22506454] = 3

predictions= predictions.astype(int)
pd.Series(predictions).value_counts(normalize=True)

3    0.412
2    0.270
0    0.167
1    0.151
dtype: float64

In [32]:
# #convert test predictions to opt R

# # optR = joblib.load(model_path+'optR_oof.pkl')
# optR = joblib.load(model_path+'optR_fulltrain.pkl')
# # optR = joblib.load(model_path+'optR_oof_removedfeats.pkl')
# # optR = joblib.load(model_path+'optR_oof_blend.pkl')
# # optR = joblib.load(model_path+'optR_fulltrain_blend.pkl')
# # optR = joblib.load(model_path+'optR_fulltrain_removedfeats.pkl')

# coefficients = optR.coefficients()
# print(coefficients)
# predictions = optR.predict(y_pred, coefficients).astype(int)
# # predictions = [item for sublist in predictions for item in sublist]
# pd.Series(predictions).value_counts(normalize=True)

In [33]:
# predictions_blend = predictions.copy()

In [34]:
# diff = predictions - predictions_ref
# print(diff[diff!=0].shape)
# print(diff[(diff==1) ].shape)
# print(diff[(diff==-1)].shape)
# print(np.unique(diff[diff!=0]))

# print(np.where(diff==-1))
# print(np.where(diff==1))

In [35]:
# print(coefficients)
# print("Test Predicted Counts = ", Counter(np.array(predictions)))

In [36]:
%%time
sample_submission['accuracy_group'] = predictions
sample_submission.to_csv('submission.csv', index = False)
print(sample_submission['accuracy_group'].value_counts(normalize = True))

3    0.412
2    0.270
0    0.167
1    0.151
Name: accuracy_group, dtype: float64
CPU times: user 2.13 s, sys: 280 ms, total: 2.41 s
Wall time: 4.84 s


In [37]:
sample_submission.head()

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3
